### 패키지 설치

In [1]:
# %pip install pymupdf
# %pip install pdfplumber

### 라이브러리 Import

In [2]:
import fitz  # PyMuPDF
import re
import json

### 변수 정의

In [3]:
# pdf 파일 경로
PDF_PATHS = [
    "../data/car_to_pedestrian_ex.pdf",
    "../data/car_to_car_ex.pdf",
    "../data/car_to_mobility_ex.pdf",
]
# 출력 파일 경로 정의
OUTPUT_PATH = "../metadata/law_metadata.json"

# 지울 단어 정의
DELETE_WORD = [
    "제1장. 자동차와 보행자의 사고",
    "제2장. 자동차와 자동차(이륜차 포함)의 사고",
    "제3장. 자동차와 자전거(농기계 포함)의 사고",
    "목차",
]

### 함수 정의

In [4]:
# pdf파일의 특정 키워드의 색상정보를 찾는 함수
def extract_color(pdf_paths:list, keywords:list) -> list: 
    # 찾을 단어의 색상 list
    extract_color = []

    for pdf_path in pdf_paths:
        doc = fitz.open(pdf_path)

        for page_num in range(len(doc)):
            page = doc[page_num]
            blocks = page.get_text("dict")["blocks"]
            for block in blocks:
                if block["type"] == 0:  # 텍스트 블록
                    for line in block["lines"]:
                        for span in line["spans"]:
                            if (span["text"] in keywords) and (span["color"] not in extract_color):
                                extract_color.append(span["color"]) # 색상 (int값, RGB 변환 가능)
    
    for color in extract_color:
        print(color)

    return extract_color

# target_color = extract_color(PDF_PATHS, SEARCH_WORD)

In [33]:
# 판례 추출
def extract_precedent(pdf_paths):
    precedents=[]
    check_section = False
    content_section = False

    for pdf_path in pdf_paths:        
        doc = fitz.open(pdf_path)

        for page in doc:
            blocks = page.get_text("dict")["blocks"]

            for block in blocks:

                if block["type"] == 0:  # 텍스트 블록만 처리
                    check_section = False

                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"]

                            # 1. '⊙'로 섹션 시작 감지
                            if text.strip() == '⊙':
                                court = ""
                                case_id = ""
                                content = ""
                                content_section = False
                                check_section = True
                                continue

                            # 2. 법원명 및 사건ID 추출
                            if check_section:
                                token = text.strip().split()
                                if (len(token) > 0) and ("법원" in token[0]) and (token[-1] == "판결"):
                                    court = token[0].strip()
                                    case_id = token[-2].strip()
                                    content_section = True
                                    continue

                            # 3. 내용 수집 (탭 문자 또는 빈 줄 전까지)
                            if content_section:
                                precedent = {}
                                if (text.strip() == ''):
                                    # 내용 종료 및 저장
                                    precedent["court"] = court
                                    precedent["case_id"] = case_id
                                    
                                    for del_keyword in DELETE_WORD:
                                        content = content.replace(del_keyword, '')
                                    precedent["content"] = content

                                    is_duplicated = False
                                    for tmp in precedents:
                                        if (tmp["court"] == court) and (tmp["case_id"] == case_id):
                                            is_duplicated = True
                                            break
                                            
                                    if not is_duplicated:
                                        precedents.append(precedent)

                                    check_section = False
                                    content_section = False
                                else:
                                    content += text.replace('\n', ' ')

    return precedents

# 실행 및 저장
precedents = extract_precedent(PDF_PATHS)
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(precedents, f, ensure_ascii=False, indent=2)

In [34]:
doc = fitz.open(PDF_PATHS[0])

all_text = ""
for page in doc:
    all_text += page.get_text() + "\n"  # 각 페이지의 텍스트를 이어붙임

doc.close()

print(all_text)

제1장. 자동차와 보행자의 사고
자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
038
4. 세부유형별 과실비율 적용기준
(1) 횡단보도 내(內) (신호등 있음)
1) 자동차 녹색신호 교차로 통과 후(後) [보1]
보1
보행자 적색신호 횡단 개시, 적색신호 충격 사고
(보) 적색에 횡단 개시, 적색에 충격 
(차) 녹색에 교차로 진입
보행자 기본 과실비율
70
과
실
비
율
조
정
예
시
①
야간·기타 시야장애
+5
②
 간선도로
+5
정지·후퇴·ㄹ자 보행
+5
③
주택·상점가·학교
-5
집단횡단
-5
④
어린이·노인·장애인
-5
어린이·노인·장애인보호구역
-15
차의 현저한 과실
-10
차의 중대한 과실
-20
보행자 급진입
비적용
⑤
보·차도 구분 없음
※사고발생, 손해확대와의 인과관계를 감안하여 기본 과실비율을 가(+), 감(-) 조정 가능합니다. 
사고 상황
	
⊙신호기가 있는 횡단보도에서 녹색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 규제를 
받는 우회전)이 보행자신호등 적색신호에 횡단보도를 건너고 있던 보행자를 보행자신호등 
적색신호가 여전히 켜져 있는 상황에서 충격한 사고이다.
기본 과실비율 해설
	
⊙도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로 이를 
위반하여 보행자신호등 적색신호에 횡단을 개시한 보행자에게 일방적으로 과실을 물어야 
할 것이나, 보행자가 상대적 교통약자인 점, 차량은 도로교통법 제27조 제1항에 따른 일시
정지 의무가 있는 점을 고려하여 보행자의 기본 과실비율을 70%로 정하였다. 
제2장. 자동차와 자동차(이륜차 포함)의 사고
제3장. 자동차와 자전거(농기계 포함)의 사고
목차

제1장. 자동차와 보행자의 사고
자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
039
수정요소(인과관계를 감안한 과실비율 조정) 해설
① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 횡단을 
예상하기 어려운 경우 보행자의 과실을 